## 导入所需库

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import lightning as L
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

## 代码

- 准备数据

In [ ]:
# ---------------------------------------------------------
# 1. 数据准备 (包含归一化优化)
# ---------------------------------------------------------
# CIFAR-10 标准均值和标准差
CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD = (0.2023, 0.1994, 0.2010)

train_transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD) # 启用归一化
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
])

# 数据集加载
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

/usr/local/miniconda3/lib/python3.13/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


- 定义model

In [ ]:
# ---------------------------------------------------------
# 2. 定义 LightningModule
# ---------------------------------------------------------

class LitConvNet(L.LightningModule):
    def __init__(self, num_classes=10, lr=0.001):
        super().__init__()
        self.save_hyperparameters()

        # 1. 增强卷积层：增加通道数（从16/32 提升至 64/128/256）
        # 通道数越多，模型“脑容量”越大，能记住的特征越丰富
        self.features = nn.Sequential(
            # 第一块：输入 3x32x32 -> 输出 64x16x16
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2), # Dropout

            # 第二块：输出 128x8x8
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.3),

            # 第三块：输出 256x4x4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.4)
        )

        # 2. 增强全连接层 (Classifier)
        # 4*4*256 = 4096
        self.classifier = nn.Sequential(
            nn.Linear(4 * 4 * 256, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5), # 高比例 Dropout 防止过拟合

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),

            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1) # 展平
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y, label_smoothing=0.1) # 引入标签平滑

        acc = (logits.argmax(1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # 使用 AdamW 配合更强的学习率策略
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=0.01)

        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.hparams.lr*10,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=0.1,
            div_factor=10,
            final_div_factor=100
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

- 训练

In [ ]:
# ---------------------------------------------------------
# 3. 训练器设置与启动
# ---------------------------------------------------------
# 实例化模型
model = LitConvNet(num_classes=10, lr=0.001)
model = torch.compile(model)

# 定义 Trainer
# Lightning 会自动处理设备映射
trainer = L.Trainer(
    max_epochs=20,
    accelerator="auto",   # 如果有 GPU 则使用
    devices=1,           # 使用 1 块卡
    precision="bf16-mixed", # 可选：开启混合精度训练，速度更快，显存占用更低
    logger=True             # Ensure tensorboard is installed first
)

# 开始训练
trainer.fit(model, train_loader)

Using bfloat16 Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/usr/local/miniconda3/lib/python3.13/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:242: Precision bf16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ features   │ Sequential │  556 K │ train │     0 │
│ 1 │ classifier │ Sequential │  4.7 M │ train │     0 │
└───┴────────────┴────────────┴────────┴───────┴───────┘

Trainable params: 5.3 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 5.3 M                                                                                                
Total estimated model params size (MB): 21                                                                         
Modules in train mode: 32                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=20` reached.


## 模型测试与保存

In [7]:
# ---------------------------------------------------------
# 4. 测试模型准确率
# ---------------------------------------------------------
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(model.device)
            labels = labels.to(model.device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

evaluate(model, test_loader)

Test Accuracy: 84.57%


In [8]:
#  保存模型
torch.save(model.state_dict(), 'model.ckpt')